In [26]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')

df_train = pd.read_csv('data/train_filtrado.csv')
df_test = pd.read_csv('data/test.csv')

[nltk_data] Downloading package stopwords to /home/camix/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def ver_info_a_filtrar(df, col):
    cant_publicaciones_columna_no_nulo = df[~df[col].isnull()].shape[0]

In [3]:
def generar_palabras_no_queridas(arr=None):
    '''Retorna un set con las palabras a excluir, arr, es un campo opcional para excluir tambien 
    las palabras en ese arreglo'''
    stop_words = set(stopwords.words('spanish'))
    puntuacion = [',', '.', '!', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '_', '\'', '\"', 
                  '(', ')', '[', ']', '{', '}', 'i', 'p', 'br', '<', '>', 'col.', '\\', '/', '$', 'm2', 
                  '..', '...', 'ii', '#', ';', ':', 'col']
    for x in puntuacion:
        stop_words.add(x)
    if arr:
        for x in arr:
            stop_words.add(x)
    return stop_words

In [4]:
def generar_palabras_mas_frecuentes(df, col, n):
    '''Esta funcion recibe un df, y genera la n palabras mas usadas la columna pasada por parametro, 
    retorna un arreglo ordenado de mas usadas a menos usadas'''
    reviews = df[col].str.cat(sep=' ')

    # Splitea los textos en palabras
    tokens = word_tokenize(reviews)

    # Fuera palabras no necesarias
    stop_words = generar_palabras_no_queridas()
    tokens = [w for w in tokens if not w in stop_words]

    vocabulary = set(tokens)
    
    # Calcula y ordena por frecuencia
    frequency_dist = nltk.FreqDist(tokens)
    mas_usadas = sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[0:n]
    return mas_usadas

In [5]:
def feature_cantidad_mas_frecuentes(df, col, n):
    '''Retorna el df pasado por parametro con la columna nueva "cant_palabras_mas_frecuentes", donde dice
    el numero de palabras mas frecuentes presentes en la columna pasada por parametro. n es la cantidad 
    palabras mas frecuentes a usar. Por ejemplo: `feature_cantidad_mas_frecuentes(df, 'titulo', 50)` genera 
    columna nueva `cant_palabras_mas_frecuentes_titulo`, donde se indica cuantas palabras frecuentes aparecen
    en el titulo, siendo las palabras mas frecuentes, la n palabras mas usadas entre todas las columnas 
    pasadas por parametro'''
    
    ver_info_a_filtrar(df, col)
    
    arr_mas_frecuentes = generar_palabras_mas_frecuentes(df, col, n)
    set_mas_frecuentes = set(arr_mas_frecuentes)
    
    def contador_mas_frecuentes(texto):
        contador = 0
        palabras_del_df = word_tokenize(texto)
        for palabra in palabras_del_df:
            if palabra in set_mas_frecuentes:
                contador = contador + 1
        return contador
    
    df['cant_palabras_mas_frecuentes_' + col] = df[~df[col].isnull()][col].apply(contador_mas_frecuentes)
    df['cant_palabras_mas_frecuentes_' + col] = df['cant_palabras_mas_frecuentes_' + col].fillna(0)
    
    return df

In [6]:
def ejemplo():
    # Ejemplo de uso de las palabras mas frecuentes
    df_palabras = feature_cantidad_mas_frecuentes(df_train, 'titulo', 200)
    df_palabras[['id', 'titulo', 'cant_palabras_mas_frecuentes_titulo']].head()

In [7]:
def generar_palabras_menos_frecuentes(df, col, n):
    '''Esta funcion recibe un df, y genera la n palabras menos usadas la columna pasada por parametro, 
    retorna un arreglo ordenado de menos usadas a menos usadas'''
    reviews = df[col].str.cat(sep=' ')

    # Splitea los textos en palabras
    tokens = word_tokenize(reviews)

    # Fuera palabras no necesarias
    stop_words = generar_palabras_no_queridas()
    tokens = [w for w in tokens if not w in stop_words]

    vocabulary = set(tokens)
    
    # Calcula y ordena por frecuencia
    frequency_dist = nltk.FreqDist(tokens)
    menos_usadas = sorted(frequency_dist, key=frequency_dist.__getitem__, reverse=False)[0:n]
    return menos_usadas

In [8]:
def feature_cantidad_menos_frecuentes(df, col, n):
    '''Retorna el df pasado por parametro con la columna nueva "cant_palabras_menos_frecuentes", donde dice
    el numero de palabras menos frecuentes presentes en la columna pasada por parametro. n es la cantidad 
    palabras menos frecuentes a usar. Por ejemplo: `cant_palabras_menos_frecuentes(df, 'titulo', 50)` genera 
    columna nueva `cant_palabras_menos_frecuentes_titulo`, donde se indica cuantas palabras menos frecuentes aparecen
    en el titulo, siendo las palabras menos frecuentes, la n palabras menos usadas entre todas las columnas 
    pasadas por parametro'''
    
    ver_info_a_filtrar(df, col)
    
    arr_menos_frecuentes = generar_palabras_menos_frecuentes(df, col, n)
    set_menos_frecuentes = set(arr_menos_frecuentes)
    
    def contador_menos_frecuentes(texto):
        contador = 0
        palabras_del_df = word_tokenize(texto)
        for palabra in palabras_del_df:
            if palabra in set_menos_frecuentes:
                contador = contador + 1
        return contador
    
    df['cant_palabras_menos_frecuentes_' + col] = df[~df[col].isnull()][col].apply(contador_menos_frecuentes)
    df['cant_palabras_menos_frecuentes_' + col] = df['cant_palabras_menos_frecuentes_' + col].fillna(0)
    
    return df

In [9]:
def ejemplo():
    # Ejemplo de uso de las palabras menos frecuentes
    df_palabras = feature_cantidad_menos_frecuentes(df_train, 'titulo', 200)
    df_palabras[['id', 'titulo', 'cant_palabras_menos_frecuentes_titulo']].head()

In [10]:
def palabras_mas_usadas_en_mas_caros_baratos(df, col, n_palabras, n_caros_baratos, df_test=None, mirar_mas_caros=True):
    """Busca los n_caros mas caros del df, y verifica las n_palabras mas usadas."""
    """Para cada texto de la columna col, cuenta cuantas de estas palabras contiene"""
    """Si se pasa un df_test, hace la cuenta teniendo en cuenta las palabras obtenidas en df"""
    """Si mirar_mas_caros es True, va a buscar las palabras mas frecuentos de los mas caros, si fuera False,
    buscaria las palabras mas frecuentes de los mas baratos"""
    
    info = 'CAROS' if mirar_mas_caros else 'BARATOS'
    
    ver_info_a_filtrar(df, col)
    df_busqueda = df.nlargest(n_caros_baratos, 'precio') if mirar_mas_caros else df.nsmallest(n_caros_baratos, 'precio')
    arr_mas_frecuentes = generar_palabras_mas_frecuentes(df_busqueda, col, n_palabras)
    set_mas_frecuentes = set(arr_mas_frecuentes)

    def contador_mas_frecuentes(texto):
        contador = 0
        palabras_del_df = word_tokenize(texto)
        for palabra in palabras_del_df:
            if palabra in set_mas_frecuentes:
                contador = contador + 1
        return contador
    
    df['palabras_mas_frecuentes_' + info + '_' + col] = df[~df[col].isnull()][col].apply(contador_mas_frecuentes)
    df['palabras_mas_frecuentes_' + info + '_' + col] = df['palabras_mas_frecuentes_' + info + '_' + col].fillna(0)
    
    if df_test is None:
        return df, df_test
    
    df_test['palabras_mas_frecuentes_' + info + '_' + col] = df_test[~df_test[col].isnull()][col].apply(contador_mas_frecuentes)
    df_test['palabras_mas_frecuentes_' + info + '_' + col] = df_test['palabras_mas_frecuentes_' + info + '_' + col].fillna(0)
    
    return df, df_test

In [11]:
def ejemplo():
    df = df_train
    df_aut = df_test
    palabras_mas_usadas_en_mas_caros_baratos(df, 'titulo', 80, 200, df_aut, False)

In [12]:
#Es mas lenteja

def mas_frecuentes_caros_baratos_ohe(df, col, n_palabras, n_caros_baratos, df_test=None, mirar_mas_caros=True):
    """Busca los n_caros mas caros del df, y verifica las n_palabras mas usadas."""
    """Hace ohe de las palabras mas usadas obtenidas, sobre la columna col"""
    """Si se pasa un df_test, hace la cuenta teniendo en cuenta las palabras obtenidas en df"""
    """Si mirar_mas_caros es True, va a buscar las palabras mas frecuentos de los mas caros, si fuera False,
    buscaria las palabras mas frecuentes de los mas baratos"""
    
    def fill_nans(df, columns, value):
        for column in columns:
            df[column] = df[column].fillna(value)
        return df
    
    ver_info_a_filtrar(df, col)
    df_busqueda = df.nlargest(n_caros_baratos, 'precio') if mirar_mas_caros else df.nsmallest(n_caros_baratos, 'precio')
    arr_mas_frecuentes = generar_palabras_mas_frecuentes(df_busqueda, col, n_palabras)
    df[col] = df[col].fillna('vacio')
    cv = CountVectorizer(vocabulary=arr_mas_frecuentes)    
    r = pd.SparseDataFrame(cv.fit_transform(df[col]), df['id'], cv.get_feature_names(), default_fill_value=0)
    
    for palabra in r.columns:
        r['palabra_' + palabra] = r[palabra]
        r.drop(palabra, axis=1, inplace=True)
    columnas = list(r.columns)
    
    r = r.reset_index()
    df_merge = df.merge(r, on='id')
    df_merge = fill_nans(df_merge, columnas, 0)
    
    if df_test is None:
        return df_merge, df_test
    
    df_test[col] = df_test[col].fillna('vacio')
    r_test = pd.SparseDataFrame(cv.fit_transform(df_test[col]), df_test['id'], cv.get_feature_names(), default_fill_value=0)
    
    for palabra in r_test.columns:
        r_test['palabra_' + palabra] = r_test[palabra]
        r_test.drop(palabra, axis=1, inplace=True)
    columnas = list(r_test.columns)
    
    r_test = r_test.reset_index()
    df_merge_aux = df_test.merge(r_test, on='id')
    df_merge_aux = fill_nans(df_merge_aux, columnas, 0)
    
    return df_merge, df_merge_aux

In [13]:
df_train_f, df_test_f = mas_frecuentes_caros_baratos_ohe(df_train, 'titulo', 10, 300, df_test,True)

In [22]:
df_train_f = df_train_f.drop(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],axis = 1,inplace = False)

df_test_f = df_test_f.drop(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos'],axis = 1,inplace = False)

In [23]:
df_train_f.to_csv('./data/titulo_train.csv',index = False)
df_test_f.to_csv('./data/titulo_test.csv',index = False)

In [28]:
df_train_f, df_test_f = mas_frecuentes_caros_baratos_ohe(df_train, 'titulo', 10, 300, df_test, False)

In [ ]:
df_train_f, df_test_f = mas_frecuentes_caros_baratos_ohe(df_train_f, 'descripcion', 10, 300, df_test_f, True)

In [ ]:
df_train_f, df_test_f = mas_frecuentes_caros_baratos_ohe(df_train_f, 'descripcion', 10, 300, df_test_f, False)

In [ ]:
df_train_f.to_csv('./data/train_titulos_caros.csv',index = False)
df_test_f.to_csv('./data/test_titulos_caros.csv',index = False)